#Importing the required library

In [ ]:
!pip install langchain
!pip install langchain_community
!pip install replicate
!pip install pypdf
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.9/357.9 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import sys

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import Chroma

# Replicate API token
os.environ['REPLICATE_API_TOKEN'] = "Your replicate api"

Creating a function to read the pdf and extract the text from it.

In [ ]:
from pypdf import PdfReader
def get_pdf_text(pdf):
  text = ""
  pdf_reader = PdfReader(pdf)
  for page in pdf_reader.pages:
    text += page.extract_text()
  return text


Storing the textual content of the input pdf in t

In [ ]:
t = get_pdf_text('./heart.pdf')

In [ ]:
print(t)

India: Noncommunicable Diseases (NCD) Country Proﬁle 
Heart disease, stroke, diabetes and cancer in India 
Impact 
 NCDs were estimated to account for 66% of all deaths in India in 2019. 
o Leading cause s of NCD mortality in 2019 were cardiovascular diseases (31%), 
chronic respiratory diseases (11%), cancers (10%), and diabetes (4% ). 
 An estimated 24% of deaths between the ages of 30 and 70 were due to NCDs in 
2019. 
 Cardiovascular diseases (CVD) are the leading cause of death in I ndia, with an 
estimated 3.8 million deaths in 2019 1. 
 The economic burden of NCDs is substantial . The accumulated loss to India on 
account of heart disease, stroke and diabetes is estimated to be $ 2.3 trillion 
between 2005 and 2015 2. 
Risk Factors 
 Prevalence of risk factors for NCDs among adults (aged 18+ years) in India 
(2019): 
o Hypertension: 25.3% of men and 22.8% of women aged 18-69 years 
o Overweight (BMI ≥ 25 kg/m2): 23.1% of men and 24.3% of women aged 18-69 years 
o Diabetes (

This code snippet splits a large text input into smaller, manageable chunks using a CharacterTextSplitter class. The splitting criteria are based on a separator (e.g., newline character) and chunk size/overlap settings. Then, it reassembles the chunks back into documents or generates new documents based on the splitting strategy

In [ ]:
#loader = PyPDFLoader('./heart.pdf')
#documents = loader.load()
documents = get_pdf_text('./heart.pdf')
# Split the documents into smaller chunks for processing
text_splitter = CharacterTextSplitter(
  separator="\n",
  chunk_size=1000,
  chunk_overlap=200,
  length_function=len
  )
chunks = text_splitter.split_text(documents)
texts= text_splitter.create_documents(chunks)
#texts = text_splitter.split_documents(documents)

Using HuggingFace Embeddings on text to and then store it in chroma vectorstore

In [ ]:
embeddings = HuggingFaceEmbeddings(
  model_name='sentence-transformers/all-MiniLM-L6-v2'
)
# embeddings = OpenAIEmbeddings(openai_api_key='Your openai key')

modules.json: 0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json: 0%|          | 0.00/122 [00:00<?, ?B/s]

sentence_bert_config.json: 0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json: 0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors: 0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json: 0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json: 0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json: 0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
vectorstore = Chroma.from_documents(texts, embeddings, persist_directory='./chroma')

In [ ]:
vectorstore.persist()

Using LLama-3 8B model as the large language model and the ConversationalRetrievalChain as the retrival chain

In [ ]:
from langchain_community.llms import Replicate

llm = Replicate(
  model="meta/meta-llama-3-8b",
  input={
    "temperature":0.6,
    "prompt" : "You are an helpful AI.You do not generate further questions and helpful answer of them",
    "max_tokens":2000,
    "system_prompt": "You are an helpful AI.You do not generate further questions and helpful answer of them",
  }
)

qa_chain = ConversationalRetrievalChain.from_llm(
  llm,
  vectorstore.as_retriever(search_kwargs={'k': 2}),  # the retrieved document from the pdf can be used to answer the prompts . k means the retrieval system will find and return the two most relevant documents (based on some similarity metric) to the question posed by the LLM.
  return_source_documents=True # returns the source doucment also
)

In [ ]:
chat_history = []  #creating the chat history
while True:
    query = input('Prompt: ')  # taking the prompt as input
    if query.lower() in ["exit", "quit", "q"]:
        print('Exiting')
        sys.exit()
    result = qa_chain({'question': query, 'chat_history': chat_history})  # retrival chain to answer our prompt
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))  # appending the chat history